In [1]:
import pandas as pd
import numpy as np

In [2]:
companies = pd.read_csv('data/company.csv', engine="pyarrow", sep=",")

In [3]:
compras_bolsa = pd.read_excel('data/Compras_Bolsa_Nacional_(kWh)_2024.xlsx', engine="openpyxl", skiprows=2)
compras_contratos = pd.read_excel('data/Compras_Contrato_(kWh)_2024.xlsx', engine="openpyxl", skiprows=2)


In [4]:
ccompras_contratos_ = compras_contratos.fillna(0)
ccompras_contratos_ = ccompras_contratos_.drop("Version", axis=1)
ccompras_contratos_ = ccompras_contratos_.set_index(["Fecha", "Código Agente","Mercado"]).stack().reset_index()
ccompras_contratos_.columns = ["date", "agent_code", "mercado","hour", "compras_contrato"]
ccompras_contratos_ = ccompras_contratos_.astype({"hour": int, "compras_contrato":float})
ccompras_contratos_ = ccompras_contratos_.merge(
    companies,
    left_on="agent_code",
    right_on="Values_Code",
    how="left"
)
ccompras_contratos_ = ccompras_contratos_.query("mercado == 'REGULADO'")
ccompras_contratos_["date"] = pd.to_datetime(ccompras_contratos_["date"])

In [5]:
compras_bolsa_ = compras_bolsa.fillna(0)
compras_bolsa_ = compras_bolsa_.drop("Version", axis=1)
compras_bolsa_ = compras_bolsa_.set_index(["Fecha", "Código Agente",]).stack().reset_index()
compras_bolsa_.columns = ["date", "agent_code", "hour", "compras_bolsa"]
compras_bolsa_ = compras_bolsa_.astype({"hour": int, "compras_bolsa":float})
compras_bolsa_ = compras_bolsa_.merge(
    companies,
    left_on="agent_code",
    right_on="Values_Code",
    how="left"
)

compras_bolsa_["date"] = pd.to_datetime(compras_bolsa_["date"])

In [6]:
ccompras_contratos_bolsa = ccompras_contratos_.merge(
    compras_bolsa_[["date", "agent_code", "hour", "compras_bolsa"]],
    on=["date", "agent_code", "hour"],
    how="left"
)
ccompras_contratos_bolsa["datetime"] = ccompras_contratos_bolsa["date"] + pd.to_timedelta(ccompras_contratos_bolsa["hour"], unit="h")
ccompras_contratos_bolsa

,date,agent_code,mercado,hour,compras_contrato,Values_Code,Values_Name,Values_Activity,compras_bolsa,datetime
0,2024-01-01,ASCC,REGULADO,0,1392.89,ASCC,A.S.C. INGENIERIA S.A. E.S.P.,COMERCIALIZACIÓN,NaN,2024-01-01 00:00:00
1,2024-01-01,ASCC,REGULADO,1,1355.81,ASCC,A.S.C. INGENIERIA S.A. E.S.P.,COMERCIALIZACIÓN,NaN,2024-01-01 01:00:00
2,2024-01-01,ASCC,REGULADO,2,1331.46,ASCC,A.S.C. INGENIERIA S.A. E.S.P.,COMERCIALIZACIÓN,NaN,2024-01-01 02:00:00
3,2024-01-01,ASCC,REGULADO,3,1351.72,ASCC,A.S.C. INGENIERIA S.A. E.S.P.,COMERCIALIZACIÓN,NaN,2024-01-01 03:00:00
4,2024-01-01,ASCC,REGULADO,4,1363.08,ASCC,A.S.C. INGENIERIA S.A. E.S.P.,COMERCIALIZACIÓN,NaN,2024-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...
371731,2024-11-21,TPLC,REGULADO,19,1475.82,TPLC,TERPEL ENERGÍA S.A.S. E.S.P.,COMERCIALIZACIÓN,3568.61,2024-11-21 19:00:00
371732,2024-11-21,TPLC,REGULADO,20,1370.82,TPLC,TERPEL ENERGÍA S.A.S. E.S.P.,COMERCIALIZACIÓN,1641.22,2024-11-21 20:00:00
371733,2024-11-21,TPLC,REGULADO,21,1255.13,TPLC,TERPEL ENERGÍA S.A.S. E.S.P.,COMERCIALIZACIÓN,1613.72,2024-11-21 21:00:00
371734,2024-11-21,TPLC,REGULADO,22,1100.36,TPLC,TERPEL ENERGÍA S.A.S. E.S.P.,COMERCIALIZACIÓN,4994.91,2024-11-21 22:00:00


In [7]:
ccompras_contratos_bolsa

,date,agent_code,mercado,hour,compras_contrato,Values_Code,Values_Name,Values_Activity,compras_bolsa,datetime
0,2024-01-01,ASCC,REGULADO,0,1392.89,ASCC,A.S.C. INGENIERIA S.A. E.S.P.,COMERCIALIZACIÓN,NaN,2024-01-01 00:00:00
1,2024-01-01,ASCC,REGULADO,1,1355.81,ASCC,A.S.C. INGENIERIA S.A. E.S.P.,COMERCIALIZACIÓN,NaN,2024-01-01 01:00:00
2,2024-01-01,ASCC,REGULADO,2,1331.46,ASCC,A.S.C. INGENIERIA S.A. E.S.P.,COMERCIALIZACIÓN,NaN,2024-01-01 02:00:00
3,2024-01-01,ASCC,REGULADO,3,1351.72,ASCC,A.S.C. INGENIERIA S.A. E.S.P.,COMERCIALIZACIÓN,NaN,2024-01-01 03:00:00
4,2024-01-01,ASCC,REGULADO,4,1363.08,ASCC,A.S.C. INGENIERIA S.A. E.S.P.,COMERCIALIZACIÓN,NaN,2024-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...
371731,2024-11-21,TPLC,REGULADO,19,1475.82,TPLC,TERPEL ENERGÍA S.A.S. E.S.P.,COMERCIALIZACIÓN,3568.61,2024-11-21 19:00:00
371732,2024-11-21,TPLC,REGULADO,20,1370.82,TPLC,TERPEL ENERGÍA S.A.S. E.S.P.,COMERCIALIZACIÓN,1641.22,2024-11-21 20:00:00
371733,2024-11-21,TPLC,REGULADO,21,1255.13,TPLC,TERPEL ENERGÍA S.A.S. E.S.P.,COMERCIALIZACIÓN,1613.72,2024-11-21 21:00:00
371734,2024-11-21,TPLC,REGULADO,22,1100.36,TPLC,TERPEL ENERGÍA S.A.S. E.S.P.,COMERCIALIZACIÓN,4994.91,2024-11-21 22:00:00


In [8]:
ccompras_contratos_bolsa_agg = ccompras_contratos_bolsa.groupby(["agent_code", ccompras_contratos_bolsa["date"].dt.month]).agg({
    # "date": "min",
    "compras_bolsa": "sum",
    "compras_contrato": "sum",
}).reset_index()
ccompras_contratos_bolsa_agg["exposicion_bolsa"] = ccompras_contratos_bolsa_agg["compras_bolsa"].div(ccompras_contratos_bolsa_agg["compras_contrato"] + ccompras_contratos_bolsa_agg["compras_bolsa"])
ccompras_contratos_bolsa_agg

,agent_code,date,compras_bolsa,compras_contrato,exposicion_bolsa
0,ASCC,1,0.00,1533202.81,0.000000
1,ASCC,2,0.00,1518673.51,0.000000
2,ASCC,3,0.00,1592295.07,0.000000
3,ASCC,4,0.00,1653088.10,0.000000
4,ASCC,5,0.00,1705141.57,0.000000
...,...,...,...,...,...
519,TPLC,7,1271002.26,895683.67,0.586611
520,TPLC,8,1601538.72,895871.75,0.641280
521,TPLC,9,1696531.34,866790.42,0.661849
522,TPLC,10,1779474.86,895683.46,0.665185


# 2. Precio Bolsa

In [9]:
pb = pd.read_csv("differences_for_month_4.csv", sep=",", engine="pyarrow", parse_dates=["datetime"])

In [10]:
ccompras_contratos_bolsa_pb = ccompras_contratos_bolsa.merge(
    pb, on="datetime"
)
ccompras_contratos_bolsa_pb = ccompras_contratos_bolsa_pb.fillna(0)
ccompras_contratos_bolsa_pb

,date,agent_code,mercado,hour,compras_contrato,Values_Code,Values_Name,Values_Activity,compras_bolsa,datetime,MPO bess_preideal_resource Modelo - DERs 2025,MPO bess_preideal_resource Modelo - DERs None,MPO preideal Modelo - DERs 2025,MPO preideal Modelo - DERs None,MPO preideal XM,MPO preideal_resource XM,Difference
0,2024-04-01,ASCC,REGULADO,0,1421.04,ASCC,A.S.C. INGENIERIA S.A. E.S.P.,COMERCIALIZACIÓN,0.00,2024-04-01 00:00:00,765000.0,765000.0,765000.0,765000.0,765000,765000,0.0
1,2024-04-01,ASCC,REGULADO,1,1363.24,ASCC,A.S.C. INGENIERIA S.A. E.S.P.,COMERCIALIZACIÓN,0.00,2024-04-01 01:00:00,765000.0,765000.0,765000.0,765000.0,765000,765000,0.0
2,2024-04-01,ASCC,REGULADO,2,1402.51,ASCC,A.S.C. INGENIERIA S.A. E.S.P.,COMERCIALIZACIÓN,0.00,2024-04-01 02:00:00,765000.0,765000.0,765000.0,765000.0,765000,765000,0.0
3,2024-04-01,ASCC,REGULADO,3,1412.41,ASCC,A.S.C. INGENIERIA S.A. E.S.P.,COMERCIALIZACIÓN,0.00,2024-04-01 03:00:00,765000.0,765000.0,765000.0,765000.0,765000,765000,0.0
4,2024-04-01,ASCC,REGULADO,4,1425.17,ASCC,A.S.C. INGENIERIA S.A. E.S.P.,COMERCIALIZACIÓN,0.00,2024-04-01 04:00:00,765000.0,765000.0,765000.0,765000.0,765000,765000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33115,2024-04-30,TPLC,REGULADO,19,1475.82,TPLC,TERPEL ENERGÍA S.A.S. E.S.P.,COMERCIALIZACIÓN,4915.28,2024-04-30 19:00:00,190000.0,190000.0,220000.0,220000.0,220000,220000,30000.0
33116,2024-04-30,TPLC,REGULADO,20,1370.82,TPLC,TERPEL ENERGÍA S.A.S. E.S.P.,COMERCIALIZACIÓN,4164.83,2024-04-30 20:00:00,190000.0,190000.0,220000.0,220000.0,220000,220000,30000.0
33117,2024-04-30,TPLC,REGULADO,21,1255.13,TPLC,TERPEL ENERGÍA S.A.S. E.S.P.,COMERCIALIZACIÓN,5415.11,2024-04-30 21:00:00,115000.0,115000.0,115000.0,115000.0,115000,115000,0.0
33118,2024-04-30,TPLC,REGULADO,22,1100.36,TPLC,TERPEL ENERGÍA S.A.S. E.S.P.,COMERCIALIZACIÓN,5741.24,2024-04-30 22:00:00,110000.0,110000.0,110000.0,110000.0,110000,110000,0.0


In [86]:
ccompras_contratos_bolsa_pb["pb_agent_XM"] = ccompras_contratos_bolsa_pb["MPO preideal XM"] * ccompras_contratos_bolsa_pb["compras_bolsa"]
ccompras_contratos_bolsa_pb["pb_agent_model"] = ccompras_contratos_bolsa_pb["MPO preideal Modelo - DERs None"] * ccompras_contratos_bolsa_pb["compras_bolsa"]
ccompras_contratos_bolsa_pb["pb_agent_model_DERS"] = ccompras_contratos_bolsa_pb["MPO preideal Modelo - DERs 2025"] * ccompras_contratos_bolsa_pb["compras_bolsa"]
ccompras_contratos_bolsa_pb["pb_agent_model_bess"] = ccompras_contratos_bolsa_pb["MPO bess_preideal_resource Modelo - DERs None"] * ccompras_contratos_bolsa_pb["compras_bolsa"]
ccompras_contratos_bolsa_pb["pb_agent_model_bess_DERS"] = ccompras_contratos_bolsa_pb["MPO bess_preideal_resource Modelo - DERs 2025"] * ccompras_contratos_bolsa_pb["compras_bolsa"]
ccompras_contratos_bolsa_pb

,date,agent_code,mercado,hour,compras_contrato,Values_Code,Values_Name,Values_Activity,compras_bolsa,datetime,...,MPO preideal Modelo - DERs 2025,MPO preideal Modelo - DERs None,MPO preideal XM,MPO preideal_resource XM,Difference,pb_agent_XM,pb_agent_model,pb_agent_model_DERS,pb_agent_model_bess,pb_agent_model_bess_DERS
0,2024-04-01,ASCC,REGULADO,0,1421.04,ASCC,A.S.C. INGENIERIA S.A. E.S.P.,COMERCIALIZACIÓN,0.00,2024-04-01 00:00:00,...,765000.0,765000.0,765000,765000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0
1,2024-04-01,ASCC,REGULADO,1,1363.24,ASCC,A.S.C. INGENIERIA S.A. E.S.P.,COMERCIALIZACIÓN,0.00,2024-04-01 01:00:00,...,765000.0,765000.0,765000,765000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0
2,2024-04-01,ASCC,REGULADO,2,1402.51,ASCC,A.S.C. INGENIERIA S.A. E.S.P.,COMERCIALIZACIÓN,0.00,2024-04-01 02:00:00,...,765000.0,765000.0,765000,765000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0
3,2024-04-01,ASCC,REGULADO,3,1412.41,ASCC,A.S.C. INGENIERIA S.A. E.S.P.,COMERCIALIZACIÓN,0.00,2024-04-01 03:00:00,...,765000.0,765000.0,765000,765000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0
4,2024-04-01,ASCC,REGULADO,4,1425.17,ASCC,A.S.C. INGENIERIA S.A. E.S.P.,COMERCIALIZACIÓN,0.00,2024-04-01 04:00:00,...,765000.0,765000.0,765000,765000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33115,2024-04-30,TPLC,REGULADO,19,1475.82,TPLC,TERPEL ENERGÍA S.A.S. E.S.P.,COMERCIALIZACIÓN,4915.28,2024-04-30 19:00:00,...,220000.0,220000.0,220000,220000,30000.0,1.081362e+09,1.081362e+09,1.081362e+09,933903200.0,933903200.0
33116,2024-04-30,TPLC,REGULADO,20,1370.82,TPLC,TERPEL ENERGÍA S.A.S. E.S.P.,COMERCIALIZACIÓN,4164.83,2024-04-30 20:00:00,...,220000.0,220000.0,220000,220000,30000.0,9.162626e+08,9.162626e+08,9.162626e+08,791317700.0,791317700.0
33117,2024-04-30,TPLC,REGULADO,21,1255.13,TPLC,TERPEL ENERGÍA S.A.S. E.S.P.,COMERCIALIZACIÓN,5415.11,2024-04-30 21:00:00,...,115000.0,115000.0,115000,115000,0.0,6.227376e+08,6.227376e+08,6.227376e+08,622737650.0,622737650.0
33118,2024-04-30,TPLC,REGULADO,22,1100.36,TPLC,TERPEL ENERGÍA S.A.S. E.S.P.,COMERCIALIZACIÓN,5741.24,2024-04-30 22:00:00,...,110000.0,110000.0,110000,110000,0.0,6.315364e+08,6.315364e+08,6.315364e+08,631536400.0,631536400.0


In [87]:
result = ccompras_contratos_bolsa_pb.groupby("agent_code").agg({
    "pb_agent_XM": "sum",
    "pb_agent_model": "sum",
    "pb_agent_model_DERS": "sum",
    "pb_agent_model_bess": "sum",
    "pb_agent_model_bess_DERS": "sum",
    "compras_bolsa": "sum",
}).reset_index()

result["pb_agent_xm_ratio"] = (result["pb_agent_XM"] / result["compras_bolsa"]) * 1E-3
result["pb_agent_model_ratio"] = (result["pb_agent_model"] / result["compras_bolsa"]) * 1E-3
result["pb_agent_model_DERS_ratio"] = (result["pb_agent_model_DERS"] / result["compras_bolsa"]) * 1E-3
result["pb_agent_model_bess_ratio"] = (result["pb_agent_model_bess"] / result["compras_bolsa"]) * 1E-3
result["pb_agent_model_bess_DERS_ratio"] = (result["pb_agent_model_bess_DERS"] / result["compras_bolsa"]) * 1E-3
result

,agent_code,pb_agent_XM,pb_agent_model,pb_agent_model_DERS,pb_agent_model_bess,pb_agent_model_bess_DERS,compras_bolsa,pb_agent_xm_ratio,pb_agent_model_ratio,pb_agent_model_DERS_ratio,pb_agent_model_bess_ratio,pb_agent_model_bess_DERS_ratio
0,ASCC,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN
1,BIAC,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN
2,CASC,1.589372e+13,1.579536e+13,1.531695e+13,1.575370e+13,1.529687e+13,1.976906e+07,803.969500,798.993661,774.793676,796.886666,773.777954
3,CDNC,6.639859e+12,6.574569e+12,6.376828e+12,6.531424e+12,6.318864e+12,8.102495e+06,819.483218,811.425208,787.020215,806.100274,779.866469
4,CEOC,6.881905e+12,6.817207e+12,6.603278e+12,6.772750e+12,6.556179e+12,8.505856e+06,809.078491,801.472252,776.321418,796.245635,770.784143
5,CETC,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN
6,CHCC,1.988807e+13,1.975241e+13,1.912441e+13,1.969522e+13,1.907901e+13,2.566636e+07,774.869034,769.583482,745.115772,767.355253,743.347086
7,CHVC,3.501525e+11,3.481828e+11,3.254311e+11,3.493668e+11,3.269282e+11,6.604094e+05,530.205218,527.222689,492.771800,529.015439,495.038662
8,CMMC,1.244872e+14,1.238080e+14,1.205463e+14,1.238495e+14,1.205959e+14,1.528037e+08,814.687311,810.242273,788.896225,810.513647,789.220970
9,CNSC,2.897687e+13,2.880252e+13,2.787465e+13,2.876849e+13,2.789244e+13,3.599815e+07,804.954419,800.111113,774.335674,799.165837,774.829794


In [88]:
result_merge_bolsa_expo = result.merge(
    ccompras_contratos_bolsa_agg.query("date == 4")[["agent_code", "exposicion_bolsa"]],
    on="agent_code", how="left"
)
result_merge_bolsa_expo

,agent_code,pb_agent_XM,pb_agent_model,pb_agent_model_DERS,pb_agent_model_bess,pb_agent_model_bess_DERS,compras_bolsa,pb_agent_xm_ratio,pb_agent_model_ratio,pb_agent_model_DERS_ratio,pb_agent_model_bess_ratio,pb_agent_model_bess_DERS_ratio,exposicion_bolsa
0,ASCC,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,0.000000
1,BIAC,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,0.000000
2,CASC,1.589372e+13,1.579536e+13,1.531695e+13,1.575370e+13,1.529687e+13,1.976906e+07,803.969500,798.993661,774.793676,796.886666,773.777954,0.424565
3,CDNC,6.639859e+12,6.574569e+12,6.376828e+12,6.531424e+12,6.318864e+12,8.102495e+06,819.483218,811.425208,787.020215,806.100274,779.866469,0.130819
4,CEOC,6.881905e+12,6.817207e+12,6.603278e+12,6.772750e+12,6.556179e+12,8.505856e+06,809.078491,801.472252,776.321418,796.245635,770.784143,0.166796
5,CETC,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,0.000000
6,CHCC,1.988807e+13,1.975241e+13,1.912441e+13,1.969522e+13,1.907901e+13,2.566636e+07,774.869034,769.583482,745.115772,767.355253,743.347086,0.292718
7,CHVC,3.501525e+11,3.481828e+11,3.254311e+11,3.493668e+11,3.269282e+11,6.604094e+05,530.205218,527.222689,492.771800,529.015439,495.038662,0.334587
8,CMMC,1.244872e+14,1.238080e+14,1.205463e+14,1.238495e+14,1.205959e+14,1.528037e+08,814.687311,810.242273,788.896225,810.513647,789.220970,0.215181
9,CNSC,2.897687e+13,2.880252e+13,2.787465e+13,2.876849e+13,2.789244e+13,3.599815e+07,804.954419,800.111113,774.335674,799.165837,774.829794,0.237817


In [89]:
result_merge_bolsa_expo["Gm_xm"] = result_merge_bolsa_expo["pb_agent_xm_ratio"] *  result_merge_bolsa_expo["exposicion_bolsa"]  
result_merge_bolsa_expo["Gm_model"] = result_merge_bolsa_expo["pb_agent_model_ratio"] *  result_merge_bolsa_expo["exposicion_bolsa"]  
result_merge_bolsa_expo["Gm_model_DERs"] = result_merge_bolsa_expo["pb_agent_model_DERS_ratio"] *  result_merge_bolsa_expo["exposicion_bolsa"]  
result_merge_bolsa_expo["Gm_model_BESS"] = result_merge_bolsa_expo["pb_agent_model_bess_ratio"] *  result_merge_bolsa_expo["exposicion_bolsa"]  
result_merge_bolsa_expo["Gm_model_BESS_DERs"] = result_merge_bolsa_expo["pb_agent_model_bess_DERS_ratio"] *  result_merge_bolsa_expo["exposicion_bolsa"]  
result_merge_bolsa_expo

,agent_code,pb_agent_XM,pb_agent_model,pb_agent_model_DERS,pb_agent_model_bess,pb_agent_model_bess_DERS,compras_bolsa,pb_agent_xm_ratio,pb_agent_model_ratio,pb_agent_model_DERS_ratio,pb_agent_model_bess_ratio,pb_agent_model_bess_DERS_ratio,exposicion_bolsa,Gm_xm,Gm_model,Gm_model_DERs,Gm_model_BESS,Gm_model_BESS_DERs
0,ASCC,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
1,BIAC,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
2,CASC,1.589372e+13,1.579536e+13,1.531695e+13,1.575370e+13,1.529687e+13,1.976906e+07,803.969500,798.993661,774.793676,796.886666,773.777954,0.424565,341.337227,339.224661,328.950196,338.330104,328.518957
3,CDNC,6.639859e+12,6.574569e+12,6.376828e+12,6.531424e+12,6.318864e+12,8.102495e+06,819.483218,811.425208,787.020215,806.100274,779.866469,0.130819,107.204336,106.150192,102.957544,105.453587,102.021695
4,CEOC,6.881905e+12,6.817207e+12,6.603278e+12,6.772750e+12,6.556179e+12,8.505856e+06,809.078491,801.472252,776.321418,796.245635,770.784143,0.166796,134.951289,133.682596,129.487530,132.810816,128.563933
5,CETC,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
6,CHCC,1.988807e+13,1.975241e+13,1.912441e+13,1.969522e+13,1.907901e+13,2.566636e+07,774.869034,769.583482,745.115772,767.355253,743.347086,0.292718,226.818398,225.271219,218.109071,224.618976,217.591344
7,CHVC,3.501525e+11,3.481828e+11,3.254311e+11,3.493668e+11,3.269282e+11,6.604094e+05,530.205218,527.222689,492.771800,529.015439,495.038662,0.334587,177.399923,176.402006,164.875177,177.001838,165.633640
8,CMMC,1.244872e+14,1.238080e+14,1.205463e+14,1.238495e+14,1.205959e+14,1.528037e+08,814.687311,810.242273,788.896225,810.513647,789.220970,0.215181,175.305456,174.348967,169.755697,174.407362,169.825576
9,CNSC,2.897687e+13,2.880252e+13,2.787465e+13,2.876849e+13,2.789244e+13,3.599815e+07,804.954419,800.111113,774.335674,799.165837,774.829794,0.237817,191.431888,190.280067,184.150228,190.055264,184.267738


In [90]:
result_merge_bolsa_expo[["agent_code"]+[col for col in result_merge_bolsa_expo if "Gm" in col]].describe()

,Gm_xm,Gm_model,Gm_model_DERs,Gm_model_BESS,Gm_model_BESS_DERs
count,37.000000,37.000000,37.000000,37.000000,37.000000
mean,281.946684,280.033192,271.986053,279.265098,271.405725
std,204.636002,203.213801,198.913178,202.637721,198.167633
min,10.186998,10.143034,10.126305,10.087248,9.497130
25%,164.762633,163.815837,158.305931,163.732011,158.634993
50%,198.800132,197.413721,190.841021,196.843379,190.885443
75%,341.337227,339.224661,328.950196,338.330104,328.518957
max,809.502205,804.104661,800.282238,802.338321,794.396507


In [91]:
result_merge_bolsa_expo[["agent_code"]+[col for col in result_merge_bolsa_expo if "Gm" in col]].head()

,agent_code,Gm_xm,Gm_model,Gm_model_DERs,Gm_model_BESS,Gm_model_BESS_DERs
0,ASCC,NaN,NaN,NaN,NaN,NaN
1,BIAC,NaN,NaN,NaN,NaN,NaN
2,CASC,341.337227,339.224661,328.950196,338.330104,328.518957
3,CDNC,107.204336,106.150192,102.957544,105.453587,102.021695
4,CEOC,134.951289,133.682596,129.487530,132.810816,128.563933


In [92]:
df = pd.read_excel("data/dema_come_agent_SEME1_2024.xlsx", skiprows=2, parse_dates=["Fecha"])


In [93]:
result_merge_bolsa_expo["delta_bess"] = result_merge_bolsa_expo["Gm_model"] - result_merge_bolsa_expo["Gm_model_BESS"]


In [94]:
df_ = df.copy()
df_ = df_[
    (df_["Mercado"]=="NO REGULADO") &
    (df_["Fecha"].dt.month == 4)
]
df_ = df_.set_index(["Fecha", "Codigo Comercializador", "Mercado", "Version"]).sum(axis=1).reset_index()
df_
df_ = df_.reset_index().groupby(["Codigo Comercializador"]).agg({0: "sum"})
result_merge_bolsa_expo = result_merge_bolsa_expo.merge(
    df_.reset_index(),
    left_on="agent_code", right_on="Codigo Comercializador", how="left"
)


In [95]:
result_merge_bolsa_expo[["agent_code", "exposicion_bolsa"]]

,agent_code,exposicion_bolsa
0,ASCC,0.000000
1,BIAC,0.000000
2,CASC,0.424565
3,CDNC,0.130819
4,CEOC,0.166796
5,CETC,0.000000
6,CHCC,0.292718
7,CHVC,0.334587
8,CMMC,0.215181
9,CNSC,0.237817


In [96]:
result_merge_bolsa_expo["beneficio"] = abs(result_merge_bolsa_expo["delta_bess"] * result_merge_bolsa_expo[0]).fillna(0)
beneficio = result_merge_bolsa_expo.beneficio.sum()
print(f"Beneficio: {beneficio*1E-6:,.2f} [MCOP/mes]")

Beneficio: 256.70 [MCOP/mes]


In [97]:
"Beneficio: 23,378.46 [MCOP/mes]"


'Beneficio: 23,378.46 [MCOP/mes]'

In [98]:
# fig = px.scatter(
#     result_merge_bolsa_expo,
#     x="delta_bess",
#     y="exposicion_bolsa",
#     color="beneficio",
#     # size="beneficio",
# )
# fig.update_layout({
#     "title": "Beneficio vs Exposición Bolsa",
#     "xaxis_title": "Beneficio [MCOP/mes]",
#     "yaxis_title": "Exposición Bolsa",
# })

In [99]:
result_merge_bolsa_expo.columns.to_list()

['agent_code',
 'pb_agent_XM',
 'pb_agent_model',
 'pb_agent_model_DERS',
 'pb_agent_model_bess',
 'pb_agent_model_bess_DERS',
 'compras_bolsa',
 'pb_agent_xm_ratio',
 'pb_agent_model_ratio',
 'pb_agent_model_DERS_ratio',
 'pb_agent_model_bess_ratio',
 'pb_agent_model_bess_DERS_ratio',
 'exposicion_bolsa',
 'Gm_xm',
 'Gm_model',
 'Gm_model_DERs',
 'Gm_model_BESS',
 'Gm_model_BESS_DERs',
 'delta_bess',
 'Codigo Comercializador',
 0,
 'beneficio']